In [ ]:
import pandas as pd
import numpy as np
import altair as alt

# Standardbreite für die Diagramme
WIDTH =500 
HEIGHT=300

from calculations import calc_cfd
from calculations import calc_leadtime
from calculations import MAXDATE


## Daten aus einer Excel-Datei einlesen

In [ ]:
DATA_FILE = "Demo.xlsx"

df_excel = pd.read_excel(DATA_FILE, sheet_name="Sheet1")


## Datenstruktur anpassen

1. Nur die gewünschten Spalten übernehmen
2. Spaltennamen ändern -> hier: Bezeichner ohne Leerzeichen verwenden

In [ ]:
df_t = df_excel[
    [
        "ID",
        "Zusammenfassung",
        "Teilsystem",
        "Bereit für Entwicklung",
        "Entwicklung in Arbeit",
        "Entwicklung fertig",
        "Abnahme in Arbeit",
        "Abnahme fertig",
        "in Produktion verfügbar",
        "in Produktion verwendet",
    ]
]

df = df_t.rename(
    {
        # "ID": "",
        # "Zusammenfassung": "",
        # "Teilsystem": "",
        "Bereit für Entwicklung": "DevReady",
        "Entwicklung in Arbeit": "DevWork",
        "Entwicklung fertig": "DevFini",
        "Team Abnahme": "TA",
        "Abnahme in Arbeit": "AbnahmeBeg",
        "Abnahme fertig": "AbnahmeFini",
        "in Produktion verfügbar": "ProduktionReady",
        "in Produktion verwendet": "Used",
    },
    axis="columns",
)


## Daten bereinigen

In [ ]:
# Nicht vorhandene Termine durch "MAXDATE" ersetzen
max_date = pd.Timestamp(MAXDATE)

df["DevReady"].fillna(max_date, inplace=True)
df["DevWork"].fillna(max_date, inplace=True)
df["DevFini"].fillna(max_date, inplace=True)
df["AbnahmeBeg"].fillna(max_date, inplace=True)
df["AbnahmeFini"].fillna(max_date, inplace=True)
df["ProduktionReady"].fillna(max_date, inplace=True)


## Berechnung der Metriken

### Daten für das Cumulative Flow Diagram

In [ ]:
pd.set_option("mode.chained_assignment", None)
cfd = calc_cfd(df)
cfd


### Berechnung der Lead Time

In [ ]:
df_lt = calc_leadtime(df)
df_lt.tail(5)


In [ ]:
def lt(df):
    display(calc_leadtime(df).describe())
    display(calc_leadtime(df).quantile([0, 0.1, 0.25, 0.5, 0.75, 0.8, 0.9, 1]))


lt(df_lt)


## Grafische Auswertung - Lead Time

### Boxplot: Leadtime zu jedem Go-live Termin

In [ ]:
def boxplot_leadtime(lead):

    return (
        alt.Chart(lead)
        .mark_boxplot(extent="min-max")
        .encode(
            alt.X(
                "ProduktionReady:T",
                title="Produktionstermin",
                timeUnit="yearmonthdate",
                axis=alt.Axis(format="%m.%Y"),
            ),
            alt.Y(
                "Leadtime:Q", title="Leadtime in Wochen", scale=alt.Scale(domain=[0, 160])
            ),
            color=alt.Color(
                "Teilsystem:O",
                scale=alt.Scale(scheme="category10"),
                legend=alt.Legend(title="Teilsystem"),
            ),
        )
        .properties(width=WIDTH, height=HEIGHT)
    )

boxplot_leadtime(df_lt)

In [ ]:
def boxplot_leadtime_withfilter(lead, produkt_filter=["Produkt A"]):
    
    return (
        alt.Chart(lead)
        .mark_boxplot(extent="min-max")
        .encode(
            alt.X(
                "ProduktionReady:T",
                title="Produktionstermin",
                timeUnit="yearmonthdate",
                axis=alt.Axis(format="%m.%Y"),
            ),
            alt.Y(
                "Leadtime:Q", title="Leadtime in Wochen", scale=alt.Scale(domain=[0, 160])
            ),
            color=alt.Color(
                "Teilsystem:O",
                scale=alt.Scale(scheme="category10"),
                legend=alt.Legend(title="Teilsystem"),
            ),
        )
        .transform_filter(
            {"not": alt.FieldOneOfPredicate(field="Teilsystem", oneOf=produkt_filter)}
        )
        .properties(width=WIDTH, height=HEIGHT)
    )

boxplot_leadtime_withfilter(df_lt, ['Produkt A', 'Produkt B'])

### Histogramm: Lead Time je Teilsystem

In [ ]:
# Anregung hierzu: https://www.digite.com/agile/lead-time-cycle-time/
def plot_lead_histogram(df):

    teilsysteme = df.Teilsystem.dropna().unique()
    selection = alt.selection_multi(fields=["Teilsystem"], bind="legend")
    chart = (
        alt.Chart(df)
        .mark_bar()
        .encode(
            alt.X(
                "Leadtime:Q",
                title="Leadtime in Wochen",
                bin=alt.Bin(extent=[0, 150], step=10),
            ),
            alt.Y("count()", title="Anzahl"),
            color=alt.Color(
                "Teilsystem",
                scale=alt.Scale(
                    domain=teilsysteme
                ),
            ),
        )
        .add_selection(selection)
        .transform_filter(selection)
        .properties(title='Histogramm der Leadtime je Teilsystem', width=500, height=300)
    )

    return chart

plot_lead_histogram(df_lt)

## Grafische Auswertung - Cumulative Flow Diagramm

In [ ]:
cfd.head(3)

In [ ]:
cfd_long = cfd.melt('Date', var_name='Typ', value_name='Anzahl')
cfd_long[cfd_long.Date == '2020-12-20']

In [ ]:
def plot_cfd(data, a_titel="Chart"):
    chart = (
        alt.Chart(data)
        .mark_area()
        .encode(
            alt.X(
                "Date:T",
                title="Datum",
                timeUnit="yearmonthdate",
                axis=alt.Axis(format="%m.%Y"),
            ),
            alt.Y("Anzahl:Q"),
            color=alt.Color(
                "Typ:O",
                scale=alt.Scale(scheme="category10"),
                legend=alt.Legend(title="Arbeitsschritt"),
            ),
            tooltip=["Typ", "Date", "Anzahl"],
        )
        .properties(title=a_titel, width=WIDTH, height=HEIGHT)
    )
    return chart.interactive()


plot_cfd(cfd_long)
